In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # visualization library
import matplotlib.pyplot as plt # visualization library
from plotly.offline import init_notebook_mode, iplot # plotly offline mode
import plotly.graph_objs as go # plotly graphical object


import src.data.timeseries_eda as eda 

init_notebook_mode(connected=True)
# ignore filters
warnings.filterwarnings("ignore") # if there is a warning after some codes, this will avoid us to see them.
plt.style.use('ggplot')

In [3]:
symbol = 'XAUUSD'

In [4]:
market_data = pd.read_csv(f'data/{symbol}_4h.csv')
market_data.head()

,open_bid,low_bid,high_bid,close_bid,volume_bid,open_ask,low_ask,high_ask,close_ask,volume_ask,avg_spread,tick_number
0,1183.410034,1183.409058,1187.402954,1186.664062,0.54846,1185.050049,1184.531982,1187.847046,1187.108032,0.64480,0.511757,1476
1,1186.684082,1184.260986,1188.281982,1186.182007,2.10916,1187.119995,1184.578003,1188.621094,1186.478027,2.17188,0.325975,4428
2,1186.260986,1184.480957,1187.481079,1185.842041,3.02581,1186.568970,1184.788940,1187.761963,1186.141968,2.69611,0.299324,6195
3,1185.881104,1180.581055,1187.940918,1182.711060,7.58135,1186.141968,1180.890991,1188.302002,1182.980957,8.02604,0.277019,13557
4,1182.851074,1167.430054,1194.330933,1187.940918,16.77736,1182.980957,1167.908936,1194.708008,1188.234985,18.69664,0.277561,33036


In [5]:
features = []
for col in market_data.columns:
    features.append(col)
feature = features[:-1]
target = features[-1:]

In [6]:
from src.data.dataloader import CreateDataset
my_dataset = CreateDataset(market_data, 10, feature, target)

train_num = int(len(market_data)*0.8)
train_data = market_data[:train_num].dropna()
test_data = market_data[train_num:].dropna()


In [7]:
from src.data.feature_engineering import PercentageFeatureAdder, BollingerBandAdder, TargetAdder, Scalar
scalar = Scalar()
train_data = scalar.fit_transform(train_data)
test_data = scalar.fit_transform(test_data)

train_data_x = train_data[feature]
train_data_y = train_data[target]

test_data_x = test_data[feature]
test_data_y = test_data[target]
test_data_x


,open_bid,low_bid,high_bid,close_bid,volume_bid,open_ask,low_ask,high_ask,close_ask,volume_ask,avg_spread
10796,0.008226,0.008213,0.008230,0.008223,0.000023,0.008227,0.008214,0.008231,0.008224,0.000020,0.000001
10797,0.008223,0.008216,0.008224,0.008217,0.000005,0.008224,0.008217,0.008226,0.008218,0.000005,0.000001
10798,0.008217,0.008206,0.008221,0.008216,0.000020,0.008218,0.008207,0.008222,0.008217,0.000020,0.000001
10799,0.008216,0.008198,0.008219,0.008209,0.000029,0.008217,0.008199,0.008221,0.008210,0.000025,0.000001
10800,0.008209,0.008195,0.008215,0.008209,0.000028,0.008210,0.008196,0.008216,0.008211,0.000023,0.000001
...,...,...,...,...,...,...,...,...,...,...,...
13491,0.008876,0.008869,0.008916,0.008911,0.000021,0.008878,0.008871,0.008918,0.008913,0.000021,0.000002
13492,0.008911,0.008888,0.008914,0.008890,0.000028,0.008913,0.008889,0.008916,0.008891,0.000014,0.000002
13493,0.008890,0.008832,0.008901,0.008853,0.000050,0.008892,0.008833,0.008902,0.008855,0.000053,0.000002
13494,0.008853,0.008850,0.008872,0.008871,0.000025,0.008855,0.008852,0.008873,0.008873,0.000016,0.000002


In [8]:
a_full_data = market_data[feature]
a_full_data = scalar.fit_transform(a_full_data)
a_full_data

,open_bid,low_bid,high_bid,close_bid,volume_bid,open_ask,low_ask,high_ask,close_ask,volume_ask,avg_spread
0,0.570064,0.570063,0.571987,0.571631,0.000264,0.570854,0.570604,0.572201,0.571845,0.000311,0.000247
1,0.571641,0.570474,0.572411,0.571399,0.001016,0.571851,0.570627,0.572574,0.571542,0.001046,0.000157
2,0.571437,0.570580,0.572025,0.571235,0.001458,0.571586,0.570728,0.572160,0.571380,0.001299,0.000144
3,0.571254,0.568701,0.572247,0.569727,0.003652,0.571380,0.568850,0.572420,0.569857,0.003866,0.000133
4,0.569795,0.562366,0.575325,0.572247,0.008082,0.569857,0.562597,0.575506,0.572388,0.009006,0.000134
...,...,...,...,...,...,...,...,...,...,...,...
13491,0.938567,0.937840,0.942816,0.942300,0.002172,0.938726,0.937994,0.942980,0.942445,0.002191,0.000163
13492,0.942291,0.939800,0.942556,0.940012,0.002990,0.942435,0.939948,0.942744,0.940181,0.001445,0.000161
13493,0.940012,0.933890,0.941173,0.936168,0.005331,0.940200,0.934049,0.941267,0.936331,0.005575,0.000162
13494,0.936154,0.935826,0.938130,0.938058,0.002644,0.936331,0.935979,0.938283,0.938206,0.001729,0.000163


In [9]:

from src.models.autoencoder import Autoencoder
a_full_data_array = a_full_data.values
autoencoder = Autoencoder(11)
autoencoder.fit(X = a_full_data_array)


Data Shuffled
Epoch 1/10 | Step 0/210 | train loss: 407.8046 | l1 loss: 406.6402 | sparsity loss: 5.8223
Epoch 1/10 | Step 50/210 | train loss: 212.9365 | l1 loss: 211.7696 | sparsity loss: 5.8347
Epoch 1/10 | Step 100/210 | train loss: 113.6182 | l1 loss: 112.3273 | sparsity loss: 6.4547
Epoch 1/10 | Step 150/210 | train loss: 53.9224 | l1 loss: 52.5102 | sparsity loss: 7.0607


Epoch 1/10 | Step 200/210 | train loss: 58.4599 | l1 loss: 56.9887 | sparsity loss: 7.3559
Data Shuffled
Epoch 2/10 | Step 0/210 | train loss: 64.8776 | l1 loss: 63.3913 | sparsity loss: 7.4313
Epoch 2/10 | Step 50/210 | train loss: 61.4929 | l1 loss: 59.9921 | sparsity loss: 7.5042
Epoch 2/10 | Step 100/210 | train loss: 59.0791 | l1 loss: 57.5309 | sparsity loss: 7.7410
Epoch 2/10 | Step 150/210 | train loss: 51.7273 | l1 loss: 50.1583 | sparsity loss: 7.8447
Epoch 2/10 | Step 200/210 | train loss: 48.6230 | l1 loss: 47.0428 | sparsity loss: 7.9013
Data Shuffled
Epoch 3/10 | Step 0/210 | train loss: 56.3139 | l1 loss: 54.6707 | sparsity loss: 8.2158
Epoch 3/10 | Step 50/210 | train loss: 44.3552 | l1 loss: 42.7505 | sparsity loss: 8.0236
Epoch 3/10 | Step 100/210 | train loss: 51.1173 | l1 loss: 49.4739 | sparsity loss: 8.2174
Epoch 3/10 | Step 150/210 | train loss: 45.5854 | l1 loss: 43.9398 | sparsity loss: 8.2280
Epoch 3/10 | Step 200/210 | train loss: 45.8264 | l1 loss: 44.1560 |